# Database Demo

Sample functionality for creating tables, inserting data and running similarity search with OgbujiPT.

__Notes:__

## Notebook requirements
```
pip install jupyter pgvector asyncpg sentence_transformers ogbujipt
```

This notebook will attempt to access a database named `demo_db` at `localhost:5432`, using the username `tester` and password `demodemodemo`. If you have a different setup, you can change the connection string in the first cell. Note: this uses different DB config than what's used for the test suite (in `test/embedding/test_pgvector.py`).

## Database requirements
This demo presumes the use of a [PostgreSQL](https://www.postgresql.org) database with the [pgvector extension](https://github.com/pgvector/pgvector)

Run the following to make a DB/PGVector instance available:
```sh
docker run --name pg_demo_db -p 5432:5432 -e POSTGRES_USER=tester -e POSTGRES_PASSWORD=demodemodemo -e POSTGRES_DB=demo_db -d ankane/pgvector
```

## Initial setup and Imports

In [80]:
%pip install jupyter pgvector asyncpg sentence_transformers ogbujipt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [55]:
import uuid

from ogbujipt.embedding.pgvector import DataDB, MessageDB
from ogbujipt.text_helper import text_split

from sentence_transformers import SentenceTransformer  # This can take a long time!

DB_NAME = 'demo_db'
HOST = 'localhost'
PORT = 8686
USER = 'tester'
PASSWORD = 'demodemodemo'

e_model = SentenceTransformer('all-MiniLM-L6-v2')  # Load the embedding model; this can also take a long time!

# Document Embedding

### Splitting the text into Chunks

In [56]:
pacer_copypasta = 'The FitnessGram™ Pacer Test is a multistage aerobic capacity test that progressively gets more difficult as it continues. The 20 meter pacer test will begin in 30 seconds. Line up at the start. The running speed starts slowly, but gets faster each minute after you hear this signal. [beep] A single lap should be completed each time you hear this sound. [ding] Remember to run in a straight line, and run as long as possible. The second time you fail to complete a lap before the sound, your test is over. The test will begin on the word start. On your mark, get ready, start.'

split_copypasta = list(text_split(pacer_copypasta, 120, separator='\\. '))
split_copypasta

['The FitnessGram™ Pacer Test is a multistage aerobic capacity test that progressively gets more difficult as it continues',
 'The 20 meter pacer test will begin in 30 seconds\\. Line up at the start',
 'The running speed starts slowly, but gets faster each minute after you hear this signal',
 '[beep] A single lap should be completed each time you hear this sound',
 '[ding] Remember to run in a straight line, and run as long as possible',
 'The second time you fail to complete a lap before the sound, your test is over\\. The test will begin on the word start',
 'On your mark, get ready, start.']

### Connecting to the database

In [57]:
pacerDB = await DataDB.from_conn_params(
    embedding_model=e_model, 
    table_name='pacer',
    db_name=DB_NAME,
    host=HOST,
    port=int(PORT),
    user=USER,
    password=PASSWORD
)

### Create Tables

In [58]:
await pacerDB.drop_table()    # Drop the table if one is found
await pacerDB.create_table()  # Create a new table

### Inserting Document

In [59]:
for index, text in enumerate(split_copypasta):          # For each line in the copypasta
    await pacerDB.insert(                               # Insert the line into the table
        content=text,                                   # The text to be embedded
        metadata={
            'title': 'Pacer Copypasta',                 # Title metadata
            'tags': ['fitness', 'pacer', 'copypasta'],  # Tag metadata
            'page_numbers': index,                      # Page number metadata
        }                               
    )

print(f'Inserted {len(pacer_copypasta)} document chunks into the table')

Inserted 576 document chunks into the table


## Similarity search Document

## Searching the document with a perfect match

In [60]:
search_string = '[beep] A single lap should be completed each time you hear this sound'
n = 3
print(f'Semantic Searching data using search string:\n"{search_string}"')

sim_search = await pacerDB.search(
    text=search_string,  # string to search by
    limit=n              # Number of results returned
)

sim_search = list(sim_search)

Semantic Searching data using search string:
"[beep] A single lap should be completed each time you hear this sound"


In [61]:
print(sim_search[0])
print(f'RETURNED TITLE:\n"{sim_search[0].metadata["title"]}"')                # Title of the first result
print(f'RETURNED TITLE:\n"{sim_search[0]["metadata"]["title"]}"')             # Dict-style access works too
print(f'RETURNED CONTENT:\n"{sim_search[0].content}"')                        # Content of the first result
print(f'RETURNED COSINE SIMILARITY:\n{sim_search[0].cosine_similarity:.2f}')  # Cosine similarity of the first result

{'cosine_similarity': 1.0, 'content': '[beep] A single lap should be completed each time you hear this sound', 'metadata': {'title': 'Pacer Copypasta', 'tags': ['fitness', 'pacer', 'copypasta'], 'page_numbers': 3}}
RETURNED TITLE:
"Pacer Copypasta"
RETURNED TITLE:
"Pacer Copypasta"
RETURNED CONTENT:
"[beep] A single lap should be completed each time you hear this sound"
RETURNED COSINE SIMILARITY:
1.00


In [62]:
print(f'RAW RETURN:')
sim_search

RAW RETURN:


[{'cosine_similarity': 1.0,
  'content': '[beep] A single lap should be completed each time you hear this sound',
  'metadata': {'title': 'Pacer Copypasta',
   'tags': ['fitness', 'pacer', 'copypasta'],
   'page_numbers': 3}},
 {'cosine_similarity': 0.6873455047607422,
  'content': 'The second time you fail to complete a lap before the sound, your test is over\\. The test will begin on the word start',
  'metadata': {'title': 'Pacer Copypasta',
   'tags': ['fitness', 'pacer', 'copypasta'],
   'page_numbers': 5}},
 {'cosine_similarity': 0.3791310417134197,
  'content': 'The running speed starts slowly, but gets faster each minute after you hear this signal',
  'metadata': {'title': 'Pacer Copypasta',
   'tags': ['fitness', 'pacer', 'copypasta'],
   'page_numbers': 2}}]

## Searching the document with a partial match

In [63]:
search_string = 'complete 1 lap'
n = 3
print(f'Semantic Searching data using search string:\n"{search_string}"')

sim_search = await pacerDB.search(
    text=search_string,  # string to search by
    limit=n              # Number of results returned
)

Semantic Searching data using search string:
"complete 1 lap"


In [64]:
row1 = next(sim_search)
print(f'RETURNED TITLE:\n"{row1.metadata["title"]}"')                # Title of the first result
print(f'RETURNED CONTENT:\n"{row1.content}"')                        # Content of the first result
print(f'RETURNED COSINE SIMILARITY:\n{row1.cosine_similarity:.2f}')  # Cosine similarity of the first result

RETURNED TITLE:
"Pacer Copypasta"
RETURNED CONTENT:
"[beep] A single lap should be completed each time you hear this sound"
RETURNED COSINE SIMILARITY:
0.53


In [65]:
# It's a generator, an we already pulled the 1st of 3 rows, so there will be 2 left
print('Remaining rows:')
list(sim_search)

Remaining rows:


[{'cosine_similarity': 0.4287309944629669,
  'content': 'The second time you fail to complete a lap before the sound, your test is over\\. The test will begin on the word start',
  'metadata': {'title': 'Pacer Copypasta',
   'tags': ['fitness', 'pacer', 'copypasta'],
   'page_numbers': 5}},
 {'cosine_similarity': 0.2411168885563878,
  'content': '[ding] Remember to run in a straight line, and run as long as possible',
  'metadata': {'title': 'Pacer Copypasta',
   'tags': ['fitness', 'pacer', 'copypasta'],
   'page_numbers': 4}}]

# Chatlog Embedding

In [66]:
abbott_and_costello = [  # Demo chatlog
    {'role': 'system', 'content': 'The user is considering becoming a ballplayer. The assistant wants to make sure they know what they\'re getting into.'},
    {'role': 'assistant', 'content': 'Strange as it may seem, they give ball players nowadays very peculiar names.'},
    {'role': 'user', 'content': 'Funny names?'},
    {'role': 'assistant', 'content': 'Nicknames, nicknames. Now, on the St. Louis team we have Who\'s on first, What\'s on second, I Don\'t Know is on third--'},
    {'role': 'user', 'content': 'That\'s what I want to find out. I want you to tell me the names of the fellows on the St. Louis team.'},
    {'role': 'assistant', 'content': "I'm telling you. Who is on first. What's on second. I Don't Know's on third--"},
    {'role': 'user', 'content': "You know the fellows' names?"},
    {'role': 'assistant', 'content': 'Yes.'},
    {'role': 'user', 'content': "Well, then who's playing first?"},
    {'role': 'assistant', 'content': 'Yes.'},
    {'role': 'user', 'content': "I mean the fellow's name on first base."},
    {'role': 'assistant', 'content': 'Who.'},
    {'role': 'user', 'content': "The fellow playin' first base."},
    {'role': 'assistant', 'content': 'Who.'},
    {'role': 'user', 'content': "The guy on first base."},
    {'role': 'assistant', 'content': 'Who is on first.'},
    {'role': 'user', 'content': "Well, what are you askin' me for?"},
    {'role': 'assistant', 'content': "I'm not asking you--I'm telling you. Who is on first."},
    {'role': 'user', 'content': "I'm asking you--who's on first?"},
    {'role': 'assistant', 'content': 'That\'s the man\'s name.'},
    {'role': 'user', 'content': "That's who's name?"},
    {'role': 'assistant', 'content': 'Yes.'},
]

### Connecting to the database

In [67]:
baseballDB = await MessageDB.from_conn_params(
    embedding_model=e_model, 
    table_name='baseball',
    user=USER,
    password=PASSWORD,
    db_name=DB_NAME,
    host=HOST,
    port=int(PORT)
)

### Create Tables

In [68]:
await baseballDB.drop_table()    # Drop the table if one is found
await baseballDB.create_table()  # Create a new table

### Inserting Chatlog

In [69]:
history_key = uuid.uuid4()        # Generate a key for the chatlog
for line in abbott_and_costello:  # For each line of dialog in the script
    await baseballDB.insert(      # Insert the message into the table
        history_key=history_key,  # The key for the chatlog
        role=line['role'],
        content=line['content'],
        metadata={'genre': 'comedy', 'year': 1938}
    )
print(f'Inserted {len(abbott_and_costello)} lines of dialog into the table with history key "{history_key}".')

Inserted 22 lines of dialog into the table with history key "de82e4e0-0e40-448b-9a18-a419a0256894".


## Searching the chatlog with a perfect match

In [70]:
search_string = 'nickname'
print(f'Semantic Searching data using search string: "{search_string}"')

sim_search = await baseballDB.search(
    history_key=history_key,
    text=search_string,
    limit=3
)

Semantic Searching data using search string: "nickname"


In [71]:
row1 = next(sim_search)
print(f'RETURNED TIMESTAMP:\n{row1.ts}')                              # Print the timestamp of the first result
print(f'RETURNED MESSAGE:\n{row1.role}: {row1.content}')  # Print the message of the first result
print(f'RETURNED COSINE SIMILARITY:\n{row1.cosine_similarity:.2f}')   # Print the cosine similarity of the first result

RETURNED TIMESTAMP:
2024-06-06 04:06:16.825910+00:00
RETURNED MESSAGE:
system: The user is considering becoming a ballplayer. The assistant wants to make sure they know what they're getting into.
RETURNED COSINE SIMILARITY:
0.11


In [72]:
# It's a generator, an we already pulled the 1st of 3 rows, so there will be 2 left
print('Remaining rows:')  
list(sim_search)

Remaining rows:


[{'ts': datetime.datetime(2024, 6, 6, 4, 6, 16, 844270, tzinfo=datetime.timezone.utc),
  'role': 'assistant',
  'content': 'Strange as it may seem, they give ball players nowadays very peculiar names.',
  'metadata': {'genre': 'comedy', 'year': 1938},
  'cosine_similarity': 0.3639257252216339},
 {'ts': datetime.datetime(2024, 6, 6, 4, 6, 16, 863395, tzinfo=datetime.timezone.utc),
  'role': 'user',
  'content': 'Funny names?',
  'metadata': {'genre': 'comedy', 'year': 1938},
  'cosine_similarity': 0.4386832118034363}]

## Searching the chatlog with a partial match

In [73]:
search_string = 'nickname'
print(f'Semantic Searching data using search string: "{search_string}"')

sim_search = await baseballDB.search(
    history_key=history_key,
    text=search_string,
    limit=3
)

Semantic Searching data using search string: "nickname"


In [74]:
row1 = next(sim_search)
print(f'RETURNED TIMESTAMP:\n{row1.ts}')                              # Print the timestamp of the first result
print(f'RETURNED MESSAGE:\n{row1.role}: {row1.content}')  # Print the message of the first result
print(f'RETURNED COSINE SIMILARITY:\n{row1.cosine_similarity:.2f}')   # Print the cosine similarity of the first result

RETURNED TIMESTAMP:
2024-06-06 04:06:16.825910+00:00
RETURNED MESSAGE:
system: The user is considering becoming a ballplayer. The assistant wants to make sure they know what they're getting into.
RETURNED COSINE SIMILARITY:
0.11


In [75]:
# It's a generator, an we already pulled the 1st of 3 rows, sso there will be 2 left
print('Remaining rows:')  
list(sim_search)

Remaining rows:


[{'ts': datetime.datetime(2024, 6, 6, 4, 6, 16, 844270, tzinfo=datetime.timezone.utc),
  'role': 'assistant',
  'content': 'Strange as it may seem, they give ball players nowadays very peculiar names.',
  'metadata': {'genre': 'comedy', 'year': 1938},
  'cosine_similarity': 0.3639257252216339},
 {'ts': datetime.datetime(2024, 6, 6, 4, 6, 16, 863395, tzinfo=datetime.timezone.utc),
  'role': 'user',
  'content': 'Funny names?',
  'metadata': {'genre': 'comedy', 'year': 1938},
  'cosine_similarity': 0.4386832118034363}]

## Retrieving the entire chatlog

In [76]:
print(f'Retreiving chatlog "{history_key}" from database')
script_from_PG = await baseballDB.get_messages(history_key=history_key)

Retreiving chatlog "de82e4e0-0e40-448b-9a18-a419a0256894" from database


In [77]:
print('RETURNED CHATLOG:')
for message in script_from_PG:
    print(f'{message["role"]}: {message["content"]}')

RETURNED CHATLOG:
assistant: Yes.
user: That's who's name?
assistant: That's the man's name.
user: I'm asking you--who's on first?
assistant: I'm not asking you--I'm telling you. Who is on first.
user: Well, what are you askin' me for?
assistant: Who is on first.
user: The guy on first base.
assistant: Who.
user: The fellow playin' first base.
assistant: Who.
user: I mean the fellow's name on first base.
assistant: Yes.
user: Well, then who's playing first?
assistant: Yes.
user: You know the fellows' names?
assistant: I'm telling you. Who is on first. What's on second. I Don't Know's on third--
user: That's what I want to find out. I want you to tell me the names of the fellows on the St. Louis team.
assistant: Nicknames, nicknames. Now, on the St. Louis team we have Who's on first, What's on second, I Don't Know is on third--
user: Funny names?
assistant: Strange as it may seem, they give ball players nowadays very peculiar names.
system: The user is considering becoming a ballplayer.

In [78]:
print('RAW RETURN:')
script_from_PG

RAW RETURN:


<generator object MessageDB.get_messages.<locals>.<genexpr> at 0x384219e40>

## (Cleanup)

In [79]:
await pacerDB.drop_table()
await baseballDB.drop_table()